# 🚗 Car Prices ETL Pipeline

## 🎯 Objective

This mini-project implements a simple yet functional **ETL process** (Extract, Transform, Load) — a key component of a **Data Engineer’s** work.

- The source data on used car prices is provided in various formats: **CSV**, **JSON**, and **XML**.  
- The task is to extract the data, convert it into a unified tabular format, and prepare it for saving in a final CSV file.

---

## ⚙️ Process Overview

- **Extract**  
  Automatic collection of data from all `.csv`, `.json`, and `.xml` files in the working directory.

- **Transform**  
  Data standardization: applying proper data types and rounding price values.

- **Load**  
  Saving the final DataFrame into `transformed_data.csv`.  
  Logging each processing step into `log_file.txt`.

---

## 💡 Practical Relevance

This project demonstrates how to work with **heterogeneous data sources**, apply a **functional approach**, and use **logging** — all of which are crucial aspects of real-world ETL pipelines.

It can serve as a **basic template for future projects** involving external data sources.

---

Next comes the step-by-step implementation: importing libraries, extracting, transforming, and loading the data.


## Step 1. Importing Libraries and Setting Up Variables

To implement the ETL process in this project, we use the following standard and third-party Python libraries:

- **pandas** — a library for working with tabular data. It is used for reading CSV and JSON files, transforming data, and creating the final DataFrame.
- **glob** — a module for finding all files in a directory that match a specific pattern (e.g., `*.csv`, `*.json`, `*.xml`). It enables automatic discovery and processing of data sources.
- **xml.etree.ElementTree** — a built-in module for parsing XML files. It allows extracting data from hierarchical XML structures.
- **datetime** — a module for handling date and time. It is used to add timestamps to the log file that tracks each ETL step.

All of these libraries are included in Python’s standard distribution, except for `pandas`, which can be installed using:


In [1]:
%pip install pandas 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


The next step is to import the required libraries and set up the global variables.


In [2]:
import pandas as pd                   # Working with tabular data (DataFrame)
import glob                           # Searching for files matching a pattern (*.csv, *.json, *.xml)
import xml.etree.ElementTree as ET    # Working with XML structure
from datetime import datetime         # Adding timestamps to the log


To ensure the correct execution of the ETL script, two global file paths are defined:

- `log_file.txt` — a file for writing execution logs. It will store timestamps and each ETL stage in sequence.
- `transformed_data.csv` — the final file where the combined and transformed dataset will be saved, ready for database loading or further analysis.

These variables are used in all key functions of the project: logging, data loading, and saving results.


In [3]:
# === VARIABLES ===
log_file = "output/log_file.txt"             # File to log the progress of the ETL process
target_file = "output/
transformed_data.csv"  # Final CSV file with the transformed data


## Step 2. Logging Function

To track the progress of the ETL process, the project includes a `log_progress()` function. It writes messages to the `log_file.txt`, adding a timestamp to each entry.

This approach helps to:

- Verify which steps have already been executed  
- Debug errors or unexpected behavior  
- Maintain an execution history (e.g., when running the script on a schedule)


In [4]:
def log_progress(message):
    timestamp_format = '%Y-%b-%d-%H:%M:%S'      # Timestamp format, e.g. 2025-Jul-26-19:45:04
    now = datetime.now()                        # Get the current time
    timestamp = now.strftime(timestamp_format)  # Format the time as a string
    with open(log_file, "a") as f:              # Open the log file in append mode
        f.write(f"{timestamp},{message}\n")     # Write the timestamp and message to the file


## Step 3. Data Extraction

At this stage, data is automatically loaded from files in ***CSV***, ***JSON***, and ***XML*** formats.  
A separate function is used for each format, returning a pandas ***DataFrame*** structure.

This modular approach improves code readability and makes it easy to adapt solutions for other projects or data sources.


### 3.1. Extracting Data from CSV

The first function, `extract_from_csv()`, is responsible for extracting data from **CSV** (comma-separated values) files.

It uses `pandas.read_csv()` to load the file content into a **DataFrame** — a tabular structure that is ideal for further processing and merging of data.

This approach makes it easy to scale the process across multiple files by automating data extraction.


In [5]:
def extract_from_csv(file_to_process):
    return pd.read_csv(file_to_process)  # Reading a CSV file into a DataFrame using pandas


### 3.2. Extracting Data from JSON

The `extract_from_json()` function is used to process files in the **JSON Lines** format (one JSON object per line).

By using the `lines=True` parameter, the `pandas.read_json()` method correctly reads the data line by line into individual records. This format is commonly used for storing logs, data streams, and system exports.

The function returns a **DataFrame**, similar to the CSV data extraction.


In [6]:
def extract_from_json(file_to_process):
    # Reads a JSON file line by line in JSON Lines format (each line is a separate JSON object)
    return pd.read_json(file_to_process, lines=True)


### 3.3. Extracting Data from XML

The `extract_from_xml()` function handles reading and parsing data from **XML files**.  
Unlike CSV and JSON, XML requires a step-by-step traversal of elements.

The project uses the `ElementTree` module from Python's standard library to:

- parse the XML file;
- get the root element;
- extract values from the `<car_model>`, `<year_of_manufacture>`, `<price>`, and `<fuel>` tags inside each item.

Each data row is added to a predefined **DataFrame** with the necessary columns.  
This approach simplifies working with XML and makes the code easier to reuse for more complex XML structures in the future.


In [7]:
def extract_from_xml(file_to_process):
    # Create an empty DataFrame with predefined columns and data types
    # Using pd.Series(dtype=...) to explicitly define the structure
    dataframe = pd.DataFrame({
        "car_model": pd.Series(dtype="str"),            # string
        "year_of_manufacture": pd.Series(dtype="int"),  # integer
        "price": pd.Series(dtype="float"),              # float
        "fuel": pd.Series(dtype="str")                  # string
    })

    # Load and parse the XML file
    tree = ET.parse(file_to_process)    # create XML tree object
    root = tree.getroot()               # get root element of the XML structure

    # Iterate over each item (e.g., <car>) within the root
    for i in root:
        # Extract values from sub-elements of the current element
        car_model = i.find("car_model").text
        year_of_manufacture = int(i.find("year_of_manufacture").text)
        price = float(i.find("price").text)
        fuel = i.find("fuel").text

        # Build a dictionary from the extracted values
        row = {
            "car_model": car_model,
            "year_of_manufacture": year_of_manufacture,
            "price": price,
            "fuel": fuel
        }

        # Append the new row to the DataFrame
        dataframe.loc[len(dataframe)] = row

    # Return the final DataFrame with data extracted from XML
    return dataframe


## Step 4. Merging All Data

The `extract()` function consolidates data extraction from all supported formats: **CSV**, **JSON**, and **XML**. It also:

- creates an empty typed DataFrame with the required columns;
- uses the `glob` module to iterate through all files in the working directory;
- calls the appropriate function depending on the file format;
- merges the extracted data into a single final DataFrame;
- skips the final CSV file from processing to avoid recursion.

This approach allows automatic processing of entire folders with mixed file types without relying on specific filenames.


In [8]:
def extract():
    # Create an empty DataFrame with predefined columns and data types
    extracted_data = pd.DataFrame({
        "car_model": pd.Series(dtype="str"),               # Car brand/model
        "year_of_manufacture": pd.Series(dtype="int"),     # Year of manufacture
        "price": pd.Series(dtype="float"),                 # Price
        "fuel": pd.Series(dtype="str")                     # Fuel type
    })

    # === Extract data from CSV files ===
    for csvfile in glob.glob("datasource/*.csv"):                     # Loop through all .csv files in the current directory
        if csvfile != target_file:                         # Skip the final output file to avoid recursive processing
            new_data = extract_from_csv(csvfile)           # Extract data using extract_from_csv()
            if not new_data.empty:                         # Only process if the result is not empty
                # Append to the main DataFrame
                # ignore_index=True resets row indices (0 to N-1) after concatenation
                extracted_data = pd.concat([extracted_data, new_data], ignore_index=True)

    # === Extract data from JSON files ===
    for jsonfile in glob.glob("datasource/*.json"):                   # Loop through all .json files
        new_data = extract_from_json(jsonfile)             # Extract data from file
        if not new_data.empty:                           # Skip empty results
            extracted_data = pd.concat([extracted_data, new_data], ignore_index=True)

    # === Extract data from XML files ===
    for xmlfile in glob.glob("datasource/*.xml"):                     # Loop through all .xml files
        new_data = extract_from_xml(xmlfile)               # Extract data from file
        if not new_data.empty:                           # Skip empty results
            extracted_data = pd.concat([extracted_data, new_data], ignore_index=True)

    return extracted_data                                   # Return the merged DataFrame


## Step 5. Data Transformation

The `transform()` function performs a basic transformation:  
- **rounding prices to two decimal places** in the `price` column.

This step is important because it:

- removes unnecessary precision (e.g., values with 10 decimal places);
- converts prices into a format commonly used in reports, databases, and user interfaces.

The function returns the updated DataFrame, ready for saving.


In [9]:
def transform(data):
    # Transform the 'price' column by rounding values to two decimal places
    data['price'] = round(data['price'], 2)
    return data  # Return the transformed DataFrame


## Step 6. Saving the Result

The final stage of the ETL pipeline is saving the transformed data.

The `load_data()` function exports the final **DataFrame** to a CSV file specified by the `target_file` variable.  
The parameter `index=False` ensures that the DataFrame index is not saved as a separate column — which is useful when importing the data into a database.

The resulting `transformed_data.csv` file can be used as a ready-to-go dataset for analysis, visualization, or storage in a data warehouse.


In [10]:
def load_data(target_file, transformed_data):
    # Save the transformed data to a CSV file without including the index
    transformed_data.to_csv(target_file, index=False)


## Step 7. Running the ETL Pipeline and Logging the Execution Flow

The full ETL cycle is executed below — from logging the start to saving the final result:

1. **Initialization**: a "start" entry is written to the log
2. **Extraction**: data is loaded from all CSV, JSON, and XML files
3. **Transformation**: price values are rounded to two decimal places
4. **Loading**: the consolidated dataset is saved to a CSV file
5. **Logging**: each step is recorded with a timestamp in `log_file.txt`

Each stage is not only executed but also documented — which is important for reproducibility, debugging, and future automation.

The `print()` output is used to visually inspect the transformed data before saving.


In [11]:
# === RUNNING THE ETL PIPELINE ===
log_progress("ETL Job Started")

# Extract phase
log_progress("Extract phase Started")
extracted_data = extract()
log_progress("Extract phase Ended")

# Transform phase
log_progress("Transform phase Started")
transformed_data = transform(extracted_data)
print("Transformed Data:")
print(transformed_data)
log_progress("Transform phase Ended")

# Load phase
log_progress("Load phase Started")
load_data(target_file, transformed_data)
log_progress("Load phase Ended")

# Completion
log_progress("ETL Job Ended")


Transformed Data:
        car_model  year_of_manufacture     price    fuel
0            ritz                 2014   5000.00  Petrol
1             sx4                 2013   7089.55  Diesel
2            ciaz                 2017  10820.90  Petrol
3         wagon r                 2011   4253.73  Petrol
4           swift                 2014   6865.67  Diesel
..            ...                  ...       ...     ...
85          camry                 2006   3731.34  Petrol
86   land cruiser                 2010  52238.81  Diesel
87  corolla altis                 2012   8805.97  Petrol
88     etios liva                 2013   5149.25  Petrol
89        etios g                 2014   7089.55  Petrol

[90 rows x 4 columns]


## ✅ Summary

This mini-project implemented a complete ETL process using Python:

🔹 **Extract:** data successfully loaded from CSV, JSON, and XML files  
🔹 **Transform:** price values rounded to two decimal places  
🔹 **Load:** the combined result saved to `transformed_data.csv`  
📝 The entire execution process was logged in the `log_file.txt` file

This project demonstrates how to automate the processing of various data formats and prepare them for further analytics or data warehousing.

---

**This ETL project** is a great starting point for building more advanced ETL pipelines with other data sources.
